In [16]:
import os
import numpy as np

import pandas as pd
from sklearn import metrics

from datetime import datetime

In [17]:
df = pd.read_csv("./avatars_echOvid_train_X.csv")

In [18]:
print(list(df.head()))

['id', 'Nom du centre', 'age', 'Sexe', 'Lieu de provenance du patient', 'Echographiste', 'Tabagisme actif', 'BPCO', 'Asthme', 'Autre antecedent respiratoire', 'Hypertension arterielle', 'Cardiopathie ischemique', 'Cardiopathie rythmique', 'Diabete de type 1', 'Diabete de type 2', 'Diabetes', 'Cancer  hemopathie maligne', 'Demence', 'Statut immunodeprime', "AINS au long cours (dans le cadre d'une pathologie suivie)", 'AINS ponctuel recent (cadre des symptomatologies COVID-19 suspect avere)', 'Tension arterielle systolique (mmHg)', 'Tension arterielle diastolique (mmHg)', 'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)', 'Temperature (Celsius)', 'Confusion', 'Saturation O2', 'Date de debut de la symptomatologie', 'Zone anterieure droite haut', 'Zone anterieure droite bas', 'Zone anterieure gauche haut', 'Zone anterieure gauche bas', 'Zone posterieure droite haut', 'Zone posterieure droite bas', 'Zone posterieure gauche haut', 'Zone posterieure gauche bas', 'Oxygeno

In [20]:
from preprocess import pre_processing

data = pre_processing(df)


In [21]:
X = data[feature_cols]
y = data['Outcome J28']

In [22]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [23]:
from utils import fit, best_scores, get_pipeline, check_scores

In [24]:
from sklearn.linear_model import LogisticRegression

lr_params = {
    "logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "logisticregression__solver": ["lbfgs", "newton-cg", "sag", "saga"],
    "logisticregression__penalty": ["l2", 'none'],
    "logisticregression__max_iter": [10000],
}

model = LogisticRegression()
clf = get_pipeline(model)
lr = fit(clf, lr_params, X, y, 20)
best_scores(lr)

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('features', SimpleImputer()),
                                                ('indicators',
                                                 MissingIndicator())])),
                ('randomforestclassifier', RandomForestClassifier())])

In [25]:
# make class predictions for the testing set
y_pred_class = clf.predict(X_test)

In [26]:
print(metrics.accuracy_score(y_test, y_pred_class))

0.8484848484848485


In [27]:
y_test.value_counts()

Back home              39
Hospitalization        23
Death                   2
Intensive care unit     2
Name: Outcome J28, dtype: int64

In [28]:
print(metrics.confusion_matrix(y_test, y_pred_class))

[[34  0  5  0]
 [ 0  0  2  0]
 [ 2  0 21  0]
 [ 0  0  1  1]]


In [29]:
y_test

194          Back home
153    Hospitalization
145          Back home
8            Back home
111          Back home
            ...       
184          Back home
15     Hospitalization
178          Back home
96           Back home
18               Death
Name: Outcome J28, Length: 66, dtype: object

In [30]:
test_data = pd.read_csv("./avatars_echOvid_test_X.csv")

pre_processing(test_data)

y_pred_class = clf.predict(X_test)

result_dataFrame =pd.DataFrame({'id':test_data['id'],'prediction':y_pred_class})
result_dataFrame = result_dataFrame.set_index('id')

display(result_dataFrame)
result_dataFrame.to_csv("./result.csv", sep =';')

,prediction
id,
237,Back home
236,Hospitalization
253,Back home
92,Back home
53,Back home
...,...
151,Back home
39,Hospitalization
313,Back home
